In [2]:
using ParallelTemperingMonteCarlo
using Profile,JET


In [3]:
n_atoms = 55
ti = 750.
tf = 1200.
n_traj = 20
temp = TempGrid{n_traj}(ti,tf) 
mc_cycles = 1000 #default 20% equilibration cycles on top


mc_sample = 1  #sample every mc_sample MC cycles

#move_atom=AtomMove(n_atoms) #move strategy (here only atom moves, n_atoms per MC cycle)
displ_atom = 0.1 # Angstrom
n_adjust = 100

max_displ_atom = [0.1*sqrt(displ_atom*temp.t_grid[i]) for i in 1:n_traj]

mc_params = MCParams(mc_cycles, n_traj, n_atoms, mc_sample = mc_sample, n_adjust = n_adjust)

ensemble = NVT(n_atoms)
move_strat = MoveStrategy(ensemble)
n = 8.482
m = 4.692
evtohartree = 0.0367493
nmtobohr = 18.8973
ϵ = evtohartree*0.0370
a = 0.25*nmtobohr
C = 27.561

pot = EmbeddedAtomPotential(n,m,ϵ,C,a)
ico_13  = [[-0.0000000049,       -0.0000000044,       -0.0000000033],
[-0.0000007312,       -0.0000000014,        0.6554619119],
 [0.1811648930,      -0.5575692094,        0.2931316798],
[-0.4742970242,       -0.3445967289,        0.2931309525],
[-0.4742970303,        0.3445967144,        0.2931309494],
 [0.1811648830,        0.5575692066,        0.2931316748],
 [0.5862626299,        0.0000000022,        0.2931321262],
[-0.1811648928,       -0.5575692153,       -0.2931316813],
[-0.5862626397,       -0.0000000109,       -0.2931321327],
[-0.1811649028,        0.5575692007,       -0.2931316863],
 [0.4742970144,        0.3445967202,       -0.2931309590],
 [0.4742970205,       -0.3445967231,       -0.2931309559],
 [0.0000007214,       -0.0000000073,       -0.6554619185]]
 copperconstant = 0.36258*nmtobohr
 AtoBohr = 1.8897259886
 pos_cu13 = copperconstant*ico_13*1.5
 bc_cu13 = SphericalBC(radius=8*AtoBohr)
 start_config = Config(pos_cu13, bc_cu13)

Config{13, SphericalBC{Float64}, Float64}(StaticArraysCore.SVector{3, Float64}[[-5.03606052999e-8, -4.52217680244e-8, -3.3916326018300003e-8], [-7.5150356316912e-6, -1.4388744371399999e-8, 6.736624211084434], [1.861953810320738, -5.73051487387157, 3.012712005572341], [-4.874670455235371, -3.541653030973376, 3.01270453061964], [-4.874670517929186, 3.541652881947096, 3.0127044987588487], [1.8619537075439925, 5.730514845094081, 3.0127119541839686], [6.025416511525561, 2.26108840122e-8, 3.0127165935262603], [-1.8619538082652034, -5.73051493450985, -3.012712020988853], [-6.025416612246772, -1.1202665260590001e-7, -3.0127166603311455], [-1.8619539110419487, 5.730514784455802, -3.0127120723772256], [4.87467035451416, 3.5416529415576075, -3.0127045974245252], [4.874670417207975, -3.541652971362864, -3.012704565563734], [7.414314421091401e-6, -7.502702422230001e-8, -6.736624278917086]], SphericalBC{Float64}(228.54811596737582))

In [4]:
dist2_mat = get_distance2_mat(start_config)

vars = set_variables(start_config,dist2_mat,pot)

EmbeddedAtomVariables([1.1280632578575635e-6 0.0015564309631457593; 4.065436622202773e-7 0.0007007747303521784; … ; 4.065436622550802e-7 0.000700774730365422; 4.0654366201822755e-7 0.000700774730149579], [0.0 0.0; 0.0 0.0; … ; 0.0 0.0; 0.0 0.0])

In [5]:
state = MCState(temp.t_grid[1],temp.beta_grid[1],start_config,ensemble,pot)

MCState{Float64, 13, SphericalBC{Float64}, EmbeddedAtomVariables, NVTVariables}(750.0, 421.0333136841296, Config{13, SphericalBC{Float64}, Float64}(StaticArraysCore.SVector{3, Float64}[[-5.03606052999e-8, -4.52217680244e-8, -3.3916326018300003e-8], [-7.5150356316912e-6, -1.4388744371399999e-8, 6.736624211084434], [1.861953810320738, -5.73051487387157, 3.012712005572341], [-4.874670455235371, -3.541653030973376, 3.01270453061964], [-4.874670517929186, 3.541652881947096, 3.0127044987588487], [1.8619537075439925, 5.730514845094081, 3.0127119541839686], [6.025416511525561, 2.26108840122e-8, 3.0127165935262603], [-1.8619538082652034, -5.73051493450985, -3.012712020988853], [-6.025416612246772, -1.1202665260590001e-7, -3.0127166603311455], [-1.8619539110419487, 5.730514784455802, -3.0127120723772256], [4.87467035451416, 3.5416529415576075, -3.0127045974245252], [4.874670417207975, -3.541652971362864, -3.012704565563734], [7.414314421091401e-6, -7.502702422230001e-8, -6.736624278917086]], Sph

In [7]:
@report_opt MCRun.mc_move!(state,move_strat,pot,ensemble)

═════ 163 possible errors found ═════
┌ mc_move!(mc_state::MCState{Float64, 13, SphericalBC{Float64}, EmbeddedAtomVariables, NVTVariables}, move_strat::MoveStrategy{55}, pot::EmbeddedAtomPotential, ensemble::NVT) @ ParallelTemperingMonteCarlo.MCRun /home/grayseff/Code/ParallelTemperingMonteCarlo.jl/src/MCRun.jl:78
│┌ get_energy!(mc_state::MCState{Float64, 13, SphericalBC{Float64}, EmbeddedAtomVariables, NVTVariables}, pot::EmbeddedAtomPotential, movetype::atommove) @ ParallelTemperingMonteCarlo.MCRun /home/grayseff/Code/ParallelTemperingMonteCarlo.jl/src/MCRun.jl:41
││┌ energy_update!(trial_pos::StaticArraysCore.SVector, index::Int64, config::Config{13, SphericalBC{Float64}, Float64}, potential_variables::EmbeddedAtomVariables, dist2_mat::Matrix{Float64}, en_tot::Float64, pot::EmbeddedAtomPotential) @ ParallelTemperingMonteCarlo.EnergyEvaluation /home/grayseff/Code/ParallelTemperingMonteCarlo.jl/src/EnergyEvaluation.jl:624
│││┌ collect(itr::Base.Generator{Vector{StaticArraysCore.SVecto

In [11]:
@report_opt MCRun.acc_test!(state,ensemble,move_strat.movestrat[1])

═════ 44 possible errors found ═════
┌ acc_test!(mc_state::MCState{Float64, 13, SphericalBC{Float64}, EmbeddedAtomVariables, NVTVariables}, ensemble::NVT, movetype::atommove) @ ParallelTemperingMonteCarlo.MCRun /home/grayseff/Code/ParallelTemperingMonteCarlo.jl/src/MCRun.jl:62
│┌ swap_config!(mc_state::MCState{Float64, 13, SphericalBC{Float64}, EmbeddedAtomVariables, NVTVariables}, movetype::atommove) @ ParallelTemperingMonteCarlo.MCRun /home/grayseff/Code/ParallelTemperingMonteCarlo.jl/src/swap_config.jl:13
││┌ swap_atom_config!(mc_state::MCState{Float64, 13, SphericalBC{Float64}, EmbeddedAtomVariables, NVTVariables}, i_atom::Int64, trial_pos::StaticArraysCore.SVector) @ ParallelTemperingMonteCarlo.MCRun /home/grayseff/Code/ParallelTemperingMonteCarlo.jl/src/swap_config.jl:22
│││┌ setindex!(A::Vector{StaticArraysCore.SVector{3, Float64}}, x::StaticArraysCore.SVector, i1::Int64) @ Base ./array.jl:1021
││││┌ convert(::Type{StaticArraysCore.SVector{3, Float64}}, sa::StaticArraysCore.SVec

In [13]:
ens=NPT(10,100)
mstrat = MoveStrategy(ens)

MoveStrategy{11}(NPT(10, 10, 1, 0, 100), Any[atommove(), atommove(), atommove(), atommove(), atommove(), atommove(), atommove(), atommove(), atommove(), atommove(), volumemove()])

In [14]:
typeof(mstrat.movestrat)

Vector{Any} (alias for Array{Any, 1})